# Regression Tree
Gregory Schuit - 16636910

### Data Preprocessing

In [5]:
import numpy as np
import pandas as pd
from time import time

In [6]:
data = pd.read_csv('data/FATS_GAIA.dat')

In [7]:
data.head()

Amplitude  AndersonDarling  Autocor_length    Class       Con  \
0   1.416432         0.949482             3.0  MIRA_SR  0.000000   
1   0.443386         1.000000             1.0     RRAB  0.000000   
2   0.170099         0.875986             4.0  MIRA_SR  0.030303   
3   1.350858         0.999869             4.0  MIRA_SR  0.000000   
4   0.248472         0.999947             3.0  MIRA_SR  0.000000   

          Eta_e  FluxPercentileRatioMid20  FluxPercentileRatioMid35  \
0      5.442293                  0.243054                  0.462373   
1  12042.752633                  0.162446                  0.231268   
2    137.020266                  0.033964                  0.208724   
3     23.583559                  0.259476                  0.593270   
4    125.486491                  0.202240                  0.448444   

   FluxPercentileRatioMid50  FluxPercentileRatioMid65    ...       PeriodLS  \
0                  0.653511                  0.761790    ...     268.331543   
1                  0.517775                  0.720122    ...       0.508910   
2                  0.378623                  0.501416    ...       8.742769   
3                  0.613808                  0.817245    ...     379.949707   
4                  0.565120                  0.606539    ...     318.427795   

   Period_fit    Psi_CS   Psi_eta       Q31       Rcs      Skew  \
0    0.025528  0.304804  0.310355  1.784879  0.304804  0.011895   
1    0.012390  0.309458  0.519645  0.397205  0.258203 -0.962335   
2    0.040022  0.342646  0.700200  0.119849  0.368936  0.472161   
3    0.000843  0.349687  0.285682  1.867060  0.349687 -0.047072   
4    0.022169  0.247451  0.578087  0.273482  0.247451  0.594605   

   SlottedA_length  SmallKurtosis       Std  
0        19.899170      -1.266215  0.931676  
1         0.147949      -0.183573  0.284637  
2         0.147949       0.064528  0.089020  
3        23.597900      -1.454570  0.926506  
4         0.147949      -0.689124  0.153349  

[5 rows x 56 columns]

In [8]:
np.random.seed(123)
shuffled = data.loc[np.random.permutation(data.index)]

percentil80 = int(len(data)*0.8)
train = shuffled[:percentil80]
test = shuffled[percentil80:]

len(train), len(test)

(160535, 40134)

### Implementation

In [85]:
class Node:
    """
    EDD principal para generar el árbol de regresión.
    """
    
    def __init__(self, feature=None, division_point=None, leaf=False, data=None):
        self.left_child = None  # de la clase Node
        self.right_child = None  # de la clase Node
        self.feature = feature  # Atributo que instancian sus hijos
        self.division_point = division_point  # punto de division que separa a sus hijos
        self.leaf = leaf  # Indica si es un nodo hoja
        self.data = data  # Si es que es un nodo hoja, contiene las particiones de los datos
        
    @property
    def depth(self):
        d = 0
        if self.left_child:
            d = 1 + self.left_child.depth
        if self.right_child:
            d = max(d, 1 + self.right_child.depth)
        return d
    
    @property
    def number_of_leaves(self):
        if self.leaf:
            return 1
        return self.left_child.number_of_leaves + self.right_child.number_of_leaves
    
    @property
    def total_var(self):
        if self.leaf:
            return self.data['PeriodLS'].var(ddof=0)
        return self.left_child.total_var + self.right_child.total_var
        
    def __repr__(self):
        if not self.leaf:
            ret = "feature: {}\ndivision_point: {}\ndepth: {}\nleaves: {}".format(self.feature,
                                                                                  self.division_point,
                                                                                  self.depth,
                                                                                  self.number_of_leaves)
        else:
            ret = "Hoja\nNúmero de datos: {}\nMean: {}\nstd: {}".format(len(self.data),
                                                                        self.data.mean(),
                                                                        self.data.std(ddof=0))
        return ret

In [127]:
def best_division_point(data, feature, target):
    """
    Calcula las varianzas de manera incremental para cada dato como punto divisorio
    de los datos y entrega el index del que minimiza la varianza.
    """
    
    print("Computing best division point for feature {}...                 ".format(feature), end='\r')
    t0 = time()
    
    if len(data[feature].unique()) == 1:  # si todos los valores de feature son iguales, como podria pasar con Autocor_length
        return 0, float('inf'), 0
    
    # Ordenamos los datos según la feature
    ordered = data.sort_values(feature)
    total = len(ordered)
        
    # Valores iniciales
    mu_left = ordered[target][:1].mean()
    mu_sq_left = sum(x**2 for x in ordered[target][:1]) / 1
    var_left = mu_sq_left - mu_left**2

    mu_right = ordered[target][1:].mean()
    mu_sq_right = sum(x**2 for x in ordered[target][1:]) / (total-1)
    var_right = mu_sq_right - mu_right**2

    minVar = (var_left + (total-1)*(var_right)) / total
    best_point = 0

    # Iteramos para recalcular las varianzas según punto de división
    columna = [x for x in ordered[feature]]  # para evitar usar iloc, que es muy muy lento
    dato_anterior = columna[0]
    for i, dato in enumerate(ordered[target][1:-2]):
        # si es que el valor se repite, no se considera, para que la particion no divida datos iguales.

        mu_left = (mu_left * (i+1) + dato) / (i+2)
        mu_sq_left = (mu_sq_left * (i+1) + dato**2) / (i+2)
        var_left = mu_sq_left - mu_left**2

        mu_right = (mu_right * (total-i-1) - dato) / (total-i-2)
        mu_sq_right = (mu_sq_right * (total-i-1) - dato**2) / (total-i-2)
        var_right = mu_sq_right - mu_right**2

        # sumamos la varianza ponderada de cada partición
        var = ((i+2)*(var_left) + (total-i-2)*(var_right)) / total
        
        if var < minVar:
            if columna[i + 1] == dato_anterior:
                continue        
            minVar = var
            best_point = i  # Almacenamos el número del dato
        
        dato_anterior = columna[i + 1]
        
    
    # retornamos el index del mejor punto de división junto con la varianza obtenidax
    idxs_left = [ordered.index[idx] for idx in range(best_point + 1)]
    idxs_right = [ordered.index[idx] for idx in range(best_point + 1, len(ordered))]
    
    # print("Best division point founded for {} in {:.4f} seconds.".format(feature, time() - t0))
    return ordered.index[best_point], minVar, [idxs_left, idxs_right]

In [128]:
def prepoda(datos):
    if len(datos) < len(data)*0.05:
        return True
    return False

def fit(data, target, max_depth=None):
    """
    Función recursiva para generar el árbol de regresión óptimo.
    """
    
    # Condición de término:
    if max_depth == 0 or prepoda(data):
        hoja = Node(leaf=True)
        hoja.data = data
        return hoja
    
    t0 = time()
    print("Computing best feature... Depth: {}".format(max_depth))
    
    features = [col for col in data.columns if col != target and type(data[col].iloc[0]) != str]

    best_feature = features[0]
    best_point, minVar, best_idxs = best_division_point(data, best_feature, target)
    for feature in features[1:]:
        point, var, idxs = best_division_point(data, feature, target)
        if var < minVar:
            best_feature = feature
            best_point = point
            minVar = var
            best_idxs = idxs  # filas de la particion
    
    print("Best feature found in {:.4f} seconds: {}, division_point: {}.".format(time() - t0, best_feature, best_point))
    
    print("Creating childs...")
    root = Node(feature=best_feature, division_point=data[best_feature][best_point])
    
    left_data = data.loc[best_idxs[0]]
    right_data = data.loc[best_idxs[1]]
    
    root.left_child = fit(left_data, target, max_depth - 1 if max_depth else None)
    print("Left child ready. Current time: {:.4f} seconds".format(time() - t0))
    root.right_child = fit(right_data, target, max_depth - 1 if max_depth else None)
    print("Right child ready. Current time: {:.4f} seconds".format(time() - t0))
    
    return root   
    

In [129]:
def predict(row, tree, target='PeriodLS'):
    """
    Recorre un árbol de regresión para predecir la clase del dato entregado.
    """
    
    # Buscamos la hoja
    while not tree.leaf:
        if row[tree.feature] < tree.division_point:
            tree = tree.left_child
        else:
            tree = tree.right_child
    
    # Predecimos en base a lo que haya en la hoja
    return tree.data[target].mean()

In [130]:
def post_poda(self, full_tree):
    pass
    """
    for 
    """
    # minimize(Sum(Varianzas) + alpha*tree.number_of_leaves)

In [131]:
regression_tree = fit(train, 'PeriodLS', 20)

Computing best feature... Depth: 20
Best feature found in 44.7202 seconds: Autocor_length, division_point: 25935.                
Creating childs...
Computing best feature... Depth: 19
Best feature found in 30.5604 seconds: Eta_e, division_point: 63864.                         
Creating childs...
Computing best feature... Depth: 18
Best feature found in 4.2821 seconds: Freq1_harmonics_amplitude_0, division_point: 129286.   
Creating childs...
Computing best feature... Depth: 17
Best feature found in 3.5471 seconds: Psi_eta, division_point: 94091.                        
Creating childs...
Left child ready. Current time: 3.5666 seconds
Right child ready. Current time: 3.5667 seconds
Left child ready. Current time: 7.8751 seconds
Right child ready. Current time: 7.8751 seconds
Left child ready. Current time: 38.5887 seconds
Computing best feature... Depth: 18
Best feature found in 26.8885 seconds: Freq3_harmonics_amplitude_0, division_point: 165517.  
Creating childs...
Computing best fe

Best feature found in 6.9162 seconds: Autocor_length, division_point: 27288.                 
Creating childs...
Computing best feature... Depth: 16
Best feature found in 4.5735 seconds: Period_fit, division_point: 35350.                     
Creating childs...
Computing best feature... Depth: 15
Best feature found in 3.7735 seconds: SlottedA_length, division_point: 16632.                
Creating childs...
Left child ready. Current time: 3.7948 seconds
Computing best feature... Depth: 14
Best feature found in 3.1337 seconds: Autocor_length, division_point: 41974.                 
Creating childs...
Left child ready. Current time: 3.1511 seconds
Right child ready. Current time: 3.1512 seconds
Right child ready. Current time: 6.9464 seconds
Left child ready. Current time: 11.5437 seconds
Right child ready. Current time: 11.5438 seconds
Left child ready. Current time: 18.4950 seconds
Right child ready. Current time: 18.4951 seconds
Left child ready. Current time: 25.5417 seconds
Right ch

### Visualización

In [105]:
from graphviz import Digraph

In [155]:
def generador_ids():
    i = 0
    while True:
        yield i
        i += 1    
    
g = generador_ids()
def make_graph(tree, graph):

    text1 = '{}\n{:.5f}'.format(tree.feature, tree.division_point)
    
    if not tree.left_child.leaf:
        text_left = '{}\n{:.5f}'.format(tree.left_child.feature, tree.left_child.division_point)
        graph.edge(text1, text_left)
        make_graph(tree.left_child, graph)
    else:
        graph.edge(text1, 'hoja N{}'.format(next(g)))
    
    if not tree.right_child.leaf:
        text_right = '{}\n{:.5f}'.format(tree.right_child.feature, tree.right_child.division_point)
        graph.edge(text1, text_right)
        make_graph(tree.right_child, graph)
    else:
        graph.edge(text1, 'hoja N{}'.format(next(g)))

In [156]:
u = Digraph('regression_tree', filename='regression_tree.gv')
u.attr(size='6,6')
u.node_attr.update(color='lightblue2', style='filled')
    
make_graph(regression_tree, u)

u.view()  # LAS HOJAS NO SE MUESTRAN EN LA VISUALIZACION

'regression_tree.gv.pdf'

### Testing

In [183]:
def execute_test(test_set, tree, target='PeriodLS'):
    t0 = time()
    length = len(test_set)
    sse = 0
    for i in range(length):
        row = test_set.iloc[i]
        sse += (predict(row, tree) - row[target])**2
        if i % 10 == 0:
            print('Seconds left: {:.2f}...'.format((time()-t0)/(i+1) * (length-i-1)), end='\r')
    mse = sse / length
    print('Test finished in {:.2f} seconds.\nMSE: {:.3f}\nsqrtMSE: {:.3f}'.format(time()-t0, mse, mse**(1/2)))

In [184]:
execute_test(test, regression_tree40)

Test finished in 38.56 seconds.
MSE: 52445.189
STD: 229.009


### Explicación

Al aplicar el modelo al conjunto de test, se calculó la distancia de cada predicción al valor real, y con esto se calculó el error cuadrático medio (MSE). El MSE es una buena manera de evaluar al modelo, ya que es comparable con la varianza de los datos crudos. Por ejemplo, a continuación calculamos la dispersión de los datos sin procesar:

In [132]:
data['PeriodLS'].std()

276.8663345620412

Este valor se puede interpretar como la raíz del MSE que obtendría un modelo que simplemente predice la media de los datos, independiente del input, ya que que si la predicción es siempre el promedio, el MSE estará calculando la dispersión con respecto a la media. Si comparamos este valor con la raiz del MSE del modelo, vemos que el modelo logra acercarse más a los valores reales que si solo se adivinara con el valor de la media general. Esta información nos dice directamente que el modelo sirve de cierto modo.

Con respecto a los distintos valores que se le pueden asignar a la profundidad maxima del arbol, podemos deducir que mientras más profundidad, hay mayor overfitting, por lo que el modelo tendería a ser más preciso, pero solo hasta cierto punto de inflexión, en el cual se empezaría a sesgar el modelo "aprendiendose" el set de training. Desde este punto el testeo comenzaria a mostrar valores de MSE más altos.  